In [1]:
import requests
import random
import time
from datetime import datetime, timezone

def get_latest_block_hash():
    url = "https://blockstream.info/api/blocks/tip/hash"
    res = requests.get(url)
    return res.text if res.status_code == 200 else None

def get_block_info(block_hash):
    url = f"https://blockstream.info/api/block/{block_hash}"
    res = requests.get(url)
    return res.json() if res.status_code == 200 else None

def get_block_txids(block_hash):
    url = f"https://blockstream.info/api/block/{block_hash}/txids"
    res = requests.get(url)
    return res.json() if res.status_code == 200 else []

def get_prev_block_hash(block_hash):
    info = get_block_info(block_hash)
    if info and "prev_block" in info:
        return info["prev_block"]
    return None

def get_transaction_details(txid):
    url = f"https://blockstream.info/api/tx/{txid}"
    res = requests.get(url)
    return res.json() if res.status_code == 200 else None

def satoshi_to_btc(sat):
    return sat / 1e8

# メイン処理
print("🔄 過去2日間のブロックから日時の異なる2件を取得中...")

block_hash = get_latest_block_hash()
tx_samples = []
seen_timekeys = set()
target_count = 2
max_blocks = 288  # 約2日分

for _ in range(max_blocks):
    if not block_hash:
        break

    block_info = get_block_info(block_hash)
    if not block_info or "timestamp" not in block_info:
        block_hash = get_prev_block_hash(block_hash)
        continue

    timestamp = block_info["timestamp"]
    dt = datetime.fromtimestamp(timestamp, tz=timezone.utc)

    minute_bucket = (dt.minute // 10) * 10
    ts_key = dt.strftime('%Y-%m-%d %H:') + str(minute_bucket).zfill(2)

    if ts_key in seen_timekeys:
        block_hash = get_prev_block_hash(block_hash)
        time.sleep(0.05)
        continue

    txids = get_block_txids(block_hash)
    if txids:
        txid = random.choice(txids)
        tx = get_transaction_details(txid)
        if tx:
            tx_samples.append((dt, tx, block_info))
            seen_timekeys.add(ts_key)

    if len(tx_samples) >= target_count:
        break

    block_hash = get_prev_block_hash(block_hash)
    time.sleep(0.05)

# 出力
print(f"\n✅ {len(tx_samples)}件のトランザクションを取得しました。\n")

for i, (dt, tx, blk_info) in enumerate(tx_samples):
    from_addresses = []
    for vin in tx['vin']:
        if 'prevout' in vin and vin['prevout'] and 'scriptpubkey_address' in vin['prevout']:
            from_addresses.append(vin['prevout']['scriptpubkey_address'])

    to_addresses = []
    total_output = 0
    for vout in tx['vout']:
        if 'scriptpubkey_address' in vout:
            to_addresses.append(vout['scriptpubkey_address'])
            total_output += vout['value']

    txid = tx['txid']
    fee_btc = satoshi_to_btc(tx.get('fee', 0))
    total_btc = satoshi_to_btc(total_output)
    size = tx['size']
    block_hash = tx['status'].get('block_hash', '不明')
    block_height = tx['status'].get('block_height', '不明')
    confirmed = tx['status'].get('confirmed', False)
    confirmations = '未確認' if not confirmed else f"{blk_info['height'] - block_height + 1} 確認"

    print(f"🧾 トランザクション {i+1}: {txid}")
    print("  📅 日時（UTC）:", dt.strftime('%Y-%m-%d %H:%M:%S'))
    print("  🔗 ブロックハッシュ:", block_hash)
    print("  🧱 ブロック高さ:", block_height)
    print("  ✅ 確認状況:", "確認済み" if confirmed else "未確認", f"（{confirmations}）")
    print("  ⏱サイズ:", size, "bytes")
    print("  💰総出力金額:", total_btc, "BTC")
    print("  💸手数料:", fee_btc, "BTC")
    print("  🔸送信元アドレス:", from_addresses if from_addresses else "（コインベース or 不明）")
    print("  🔹送信先アドレス:", to_addresses)
    print()

    print("  📥 vin（入力）:")
    for vin_idx, vin in enumerate(tx['vin']):
        prevout = vin.get('prevout', {})
        addr = prevout.get('scriptpubkey_address', '不明')
        value = satoshi_to_btc(prevout.get('value', 0))
        script_type = prevout.get('scriptpubkey_type', '不明')
        txid_in = vin.get('txid', 'コインベース')
        vout_n = vin.get('vout', 'N/A')

        print(f"    [{vin_idx}] txid: {txid_in}, vout: {vout_n}")
        print(f"         アドレス: {addr}")
        print(f"         金額: {value} BTC, スクリプト種別: {script_type}")

    print("  📤 vout（出力）:")
    for vout_idx, vout in enumerate(tx['vout']):
        value = satoshi_to_btc(vout.get('value', 0))
        addr = vout.get('scriptpubkey_address', '（アドレスなし）')
        script_type = vout.get('scriptpubkey_type', '不明')
        print(f"    [{vout_idx}] 金額: {value} BTC, アドレス: {addr}, スクリプト種別: {script_type}")

    print("\n" + "-" * 80 + "\n")


🔄 過去2日間のブロックから日時の異なる2件を取得中...

✅ 1件のトランザクションを取得しました。

🧾 トランザクション 1: 856570280d46e48497cad1b038a2849b094ee4436c9e0309428b88e89466dbe9
  📅 日時（UTC）: 2025-06-11 08:43:33
  🔗 ブロックハッシュ: 00000000000000000000162fc231170ae2b482b84e0100d286bf840da0646376
  🧱 ブロック高さ: 900759
  ✅ 確認状況: 確認済み （1 確認）
  ⏱サイズ: 381 bytes
  💰総出力金額: 0.00052851 BTC
  💸手数料: 4.91e-06 BTC
  🔸送信元アドレス: ['bc1px8cvzmfy67p75j9sraq2kjrusy2xkv3uen3cntgg74a7m6mz2tss2rx397']
  🔹送信先アドレス: ['bc1p3wk57fk6qw4zd6e5ch7fwmhzap86a3mv68lh7up6z30ru7sfy6mqn85nqh', 'bc1px8cvzmfy67p75j9sraq2kjrusy2xkv3uen3cntgg74a7m6mz2tss2rx397']

  📥 vin（入力）:
    [0] txid: d7a361df807feb4910f2e882307ab91d3869809738b185035f5af2aa32350133, vout: 1
         アドレス: bc1px8cvzmfy67p75j9sraq2kjrusy2xkv3uen3cntgg74a7m6mz2tss2rx397
         金額: 0.00053342 BTC, スクリプト種別: v1_p2tr
  📤 vout（出力）:
    [0] 金額: 3.3e-06 BTC, アドレス: bc1p3wk57fk6qw4zd6e5ch7fwmhzap86a3mv68lh7up6z30ru7sfy6mqn85nqh, スクリプト種別: v1_p2tr
    [1] 金額: 0.00052521 BTC, アドレス: bc1px8cvzmfy67p75j9sraq2kjrusy2xkv3uen3c